In [371]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = r"C:\Users\Peter\Source\Repos\pandas-challenge\Resources\schools_complete.csv"
student_data_to_load = r"C:\Users\Peter\Source\Repos\pandas-challenge\Resources\students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.dtypes

Student ID        int64
student_name     object
gender           object
grade            object
school_name      object
reading_score     int64
math_score        int64
School ID         int64
type             object
size              int64
budget            int64
dtype: object

In [372]:
# total number of schools equals to the unique numbers of schools
total_schools = len(pd.unique(school_data_complete["school_name"]))

# total number of students equals to the unique numbers of student id
total_students = len(pd.unique(school_data_complete["Student ID"]))

# total budget equals to the sum of each schools budget from school data
total_budget  = school_data["budget"].sum()

# average the math score
average_math_score = school_data_complete["math_score"].mean()

# average the reading score
average_reading_score = school_data_complete["reading_score"].mean()

# to find out the % passing math, we need to find out the total number of students who pass the math, then divide it by total number of students
total_pass_math = school_data_complete.loc[school_data_complete["math_score"] >= 70, :]["Student ID"].count()
percent_passing_math = total_pass_math / total_students

# to find out the % passing reading, we need to find out the total number of students who pass the mreading, then divide it by total number of students
total_pass_reading = school_data_complete.loc[school_data_complete["reading_score"] >= 70, :]["Student ID"].count()
percent_passing_reading = total_pass_reading / total_students

# to find out the % passing overall, we need to find out the total number of students who pass the math and reading, then divide it by total number of students
total_pass_overall = school_data_complete.loc[(school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70), :]["Student ID"].count()
percent_passing_overall = total_pass_overall / total_students

# formate the results into a dictionary then into a dataframe
district_summary = {"Total Schools": total_schools,
                    'Total Students': total_students,
                    "Total Budget": total_budget,
                    "Average Math Score": '{:.2f}'.format(average_math_score),
                    "Average Reading Score": '{:.2f}'.format(average_reading_score),
                    "% Passing Math": '{:.2%}'.format(percent_passing_math),
                    "% Passing Reading": '{:.2%}'.format(percent_passing_reading),
                    "% Overall Passing": '{:.2%}'.format(percent_passing_overall),
                    }
Disctrict_summary = pd.DataFrame(district_summary, index=[0])
Disctrict_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,24649428,78.99,81.88,74.98%,85.81%,65.17%


In [378]:
# in order to form a data frame that summarizes key metris about each school, we do the same thing in district summary but for each school
# we can create a list of schools and group the school data by school names
school_list = list(pd.unique(school_data_complete["school_name"]))
groupby_school = school_data_complete.groupby(["school_name"])

# create data frame grouped by school with mean of each entries
groupby_mean = groupby_school.mean()
groupby_mean = groupby_mean.reset_index()

# create data frame grouped by school with math score greater or equal to 70
groupby_mass_pass = school_data_complete.loc[school_data_complete["math_score"] >= 70, ["school_name","math_score"]]
groupby_mass_pass = groupby_mass_pass.groupby(["school_name"])
groupby_mass_pass = groupby_mass_pass.count()
groupby_mass_pass = groupby_mass_pass.reset_index()


# create data frame grouped by school with reading score greater or equal to 70
groupby_reading_pass = school_data_complete.loc[school_data_complete["reading_score"] >= 70, ["school_name","reading_score"]]
groupby_reading_pass = groupby_reading_pass.groupby(["school_name"])
groupby_reading_pass = groupby_reading_pass.count()
groupby_reading_pass = groupby_reading_pass.reset_index()

# create data frame grouped by school with overall score greater or equal to 70
groupby_overall_pass = school_data_complete.loc[(school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70), ["school_name","math_score","reading_score"]]
groupby_overall_pass = groupby_overall_pass.groupby(["school_name"])
groupby_overall_pass = groupby_overall_pass.count()
groupby_overall_pass = groupby_overall_pass.reset_index()


# for each school in the school list, we loop through the whole datasets and perform the analysis for each school and form a list of dictionaries
school_summary = []
for school in school_list:
    school_name = school
    school_type = school_data.loc[school_data["school_name"] == school, :]["type"].values
    total_students = int(school_data.loc[school_data["school_name"] == school, :]["size"])
    Total_school_budget = int(school_data.loc[school_data["school_name"] == school, :]["budget"])
    per_student_budget = Total_school_budget / total_students
    average_math_score = groupby_mean.loc[groupby_mean["school_name"] == school, :]["math_score"]
    average_reading_score = groupby_mean.loc[groupby_mean["school_name"] == school, :]["reading_score"]
    percent_math_pass = float(groupby_mass_pass.loc[groupby_mass_pass["school_name"] == school, :]["math_score"]) / total_students
    percent_reading_pass = float(groupby_reading_pass.loc[groupby_reading_pass["school_name"] == school, :]["reading_score"]) / total_students
    percent_overall_pass = float(groupby_overall_pass.loc[groupby_overall_pass["school_name"] == school, :]["reading_score"]) / total_students
    school_summary_dict = {"School Name": school_name,
                            "School Type": school_type[0],
                            'Total Students': total_students,
                            "Total Budget": Total_school_budget,
                            "Per Student Budget": per_student_budget,
                            "Average Math Score": '{:.2f}'.format(float(average_math_score)),
                            "Average Reading Score": '{:.2f}'.format(float(average_reading_score)),
                            "% Passing Math": '{:.2f}'.format(float(percent_math_pass) * 100),
                            "% Passing Reading": '{:.2f}'.format(float(percent_reading_pass) * 100),
                            "% Overall Passing": '{:.2f}'.format(float(percent_overall_pass)* 100),
                            }
    
    school_summary.append(school_summary_dict)


# conver the list into dataframe
school_summary = pd.DataFrame(school_summary) 
school_summary.set_index("School Name")

,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Huang High School,District,2917,1910635,655.0,76.63,81.18,65.68,81.32,53.51
Figueroa High School,District,2949,1884411,639.0,76.71,81.16,65.99,80.74,53.20
Shelton High School,Charter,1761,1056600,600.0,83.36,83.73,93.87,95.85,89.89
Hernandez High School,District,4635,3022020,652.0,77.29,80.93,66.75,80.86,53.53
Griffin High School,Charter,1468,917500,625.0,83.35,83.82,93.39,97.14,90.60
Wilson High School,Charter,2283,1319574,578.0,83.27,83.99,93.87,96.54,90.58
Cabrera High School,Charter,1858,1081356,582.0,83.06,83.98,94.13,97.04,91.33
Bailey High School,District,4976,3124928,628.0,77.05,81.03,66.68,81.93,54.64
Holden High School,Charter,427,248087,581.0,83.80,83.81,92.51,96.25,89.23


In [374]:
# to get the highest perfoming schools, we sort the school summary by the overall passing 
highest_performing_school = school_summary.sort_values("% Overall Passing", ascending = False).head(5)
highest_performing_school.set_index("School Name")

,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Griffin High School,Charter,1468,917500,625.0,83.35,83.82,0.93,0.97,0.91
Wilson High School,Charter,2283,1319574,578.0,83.27,83.99,0.94,0.97,0.91
Cabrera High School,Charter,1858,1081356,582.0,83.06,83.98,0.94,0.97,0.91
Pena High School,Charter,962,585858,609.0,83.84,84.04,0.95,0.96,0.91
Thomas High School,Charter,1635,1043130,638.0,83.42,83.85,0.93,0.97,0.91


In [375]:
# to get the lowest perfoming schools, we sort the school summary by the overall passing 
lowest_performing_school = school_summary.sort_values("% Overall Passing").head(5)
lowest_performing_school.set_index("School Name")

,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Figueroa High School,District,2949,1884411,639.0,76.71,81.16,0.66,0.81,0.53
Rodriguez High School,District,3999,2547363,637.0,76.84,80.74,0.66,0.80,0.53
Huang High School,District,2917,1910635,655.0,76.63,81.18,0.66,0.81,0.54
Hernandez High School,District,4635,3022020,652.0,77.29,80.93,0.67,0.81,0.54
Johnson High School,District,4761,3094650,650.0,77.07,80.97,0.66,0.81,0.54


In [317]:
# setup a data frame with only math score, grade level and the school name
math_scores_by_grade = school_data_complete[["math_score","school_name","grade"]]
math_scores_by_grade = math_scores_by_grade.groupby(["school_name","grade"])
math_scores_by_grade = math_scores_by_grade.mean()
math_scores_by_grade = math_scores_by_grade.reset_index()

# use for loop to create a list of dictionary for each school each grade's math score
school_list = list(pd.unique(school_data_complete["school_name"]))
math_summary = []
for school in school_list:
        ninth_grade = math_scores_by_grade.loc[(math_scores_by_grade["school_name"] == school) & (math_scores_by_grade["grade"] == "9th"), :]["math_score"].values[0]
        tenth_grade = math_scores_by_grade.loc[(math_scores_by_grade["school_name"] == school) & (math_scores_by_grade["grade"] == "10th"), :]["math_score"].values[0]
        eleventh_grade = math_scores_by_grade.loc[(math_scores_by_grade["school_name"] == school) & (math_scores_by_grade["grade"] == "11th"), :]["math_score"].values[0]
        twelfth_grade = math_scores_by_grade.loc[(math_scores_by_grade["school_name"] == school) & (math_scores_by_grade["grade"] == "12th"), :]["math_score"].values[0]
        
        math_dict = {"School Name": school,
                     "9th": ninth_grade,
                     "10th": tenth_grade,
                    "11th": eleventh_grade,
                     "12th": twelfth_grade
                        }
        
        math_summary.append(math_dict)
math_summary = pd.DataFrame(math_summary)
math_summary = math_summary.set_index("School Name")
math_summary

,9th,10th,11th,12th
School Name,,,,
Huang High School,77.027251,75.908735,76.446602,77.225641
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Shelton High School,83.420755,82.917411,83.383495,83.778976
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Griffin High School,82.044010,84.229064,83.842105,83.356164
Wilson High School,83.085578,83.724422,83.195326,83.035794
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Bailey High School,77.083676,76.996772,77.515588,76.492218
Holden High School,83.787402,83.429825,85.000000,82.855422


In [318]:
# setup a data frame with only reading score, grade level and the school name
reading_scores_by_grade = school_data_complete[["reading_score","school_name","grade"]]
reading_scores_by_grade = reading_scores_by_grade.groupby(["school_name","grade"])
reading_scores_by_grade = reading_scores_by_grade.mean()
reading_scores_by_grade = reading_scores_by_grade.reset_index()

# use for loop to create a list of dictionary for each school each grade's math score
school_list = list(pd.unique(school_data_complete["school_name"]))
reading_summary = []
for school in school_list:
        ninth_grade = reading_scores_by_grade.loc[(reading_scores_by_grade["school_name"] == school) & (reading_scores_by_grade["grade"] == "9th"), :]["reading_score"].values[0]
        tenth_grade = reading_scores_by_grade.loc[(reading_scores_by_grade["school_name"] == school) & (reading_scores_by_grade["grade"] == "10th"), :]["reading_score"].values[0]
        eleventh_grade = reading_scores_by_grade.loc[(reading_scores_by_grade["school_name"] == school) & (reading_scores_by_grade["grade"] == "11th"), :]["reading_score"].values[0]
        twelfth_grade = reading_scores_by_grade.loc[(reading_scores_by_grade["school_name"] == school) & (reading_scores_by_grade["grade"] == "12th"), :]["reading_score"].values[0]
        
        reading_dict = {"School Name": school,
                     "9th": ninth_grade,
                     "10th": tenth_grade,
                    "11th": eleventh_grade,
                     "12th": twelfth_grade
                        }
        
        reading_summary.append(reading_dict)
reading_summary = pd.DataFrame(reading_summary)
reading_summary = reading_summary.set_index("School Name")
reading_summary

,9th,10th,11th,12th
School Name,,,,
Huang High School,81.290284,81.512386,81.417476,80.305983
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Shelton High School,84.122642,83.441964,84.373786,82.781671
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Griffin High School,83.369193,83.706897,84.288089,84.013699
Wilson High School,83.939778,84.021452,83.764608,84.317673
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Bailey High School,81.303155,80.907183,80.945643,80.912451
Holden High School,83.677165,83.324561,83.815534,84.698795


In [383]:
# find the maximumn and minimumn value of budget
max_budget = school_summary["Per Student Budget"].max()
min_budget = school_summary["Per Student Budget"].min()

# use the school summary but narrow it down to per student buget average math score, average reading score percent passing math percent passing reading perscent overall passing
score_by_spending = school_summary[["Per Student Budget","Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing"]]
score_by_spending["Average Math Score"] = pd.to_numeric(score_by_spending["Average Math Score"])
score_by_spending["Average Reading Score"] = pd.to_numeric(score_by_spending["Average Reading Score"])
score_by_spending["% Passing Math"] = pd.to_numeric(score_by_spending["% Passing Math"])
score_by_spending["% Passing Reading"] = pd.to_numeric(score_by_spending["% Passing Reading"])
score_by_spending["% Overall Passing"] = pd.to_numeric(score_by_spending["% Overall Passing"])



# bins for cutting
bins = [0,584.999,629.999,644.999,680]

# name for each bin
budget_range = ["< 585", "585 - 630", "630 - 645", "645 - 680"]

# put values into the dataframe
score_by_spending["Spending Ranges (Per Student)"] = pd.cut(score_by_spending["Per Student Budget"], bins, labels = budget_range)
score_by_spending = score_by_spending.drop(columns = ["Per Student Budget"])
score_by_spending = score_by_spending.groupby(["Spending Ranges (Per Student)"])
score_by_spending = score_by_spending.mean()
score_by_spending

C:\Users\Peter\AppData\Local\Temp\ipykernel_5456\1717203083.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_by_spending["Average Math Score"] = pd.to_numeric(score_by_spending["Average Math Score"])
C:\Users\Peter\AppData\Local\Temp\ipykernel_5456\1717203083.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_by_spending["Average Reading Score"] = pd.to_numeric(score_by_spending["Average Reading Score"])
C:\Users\Peter\AppData\Local\Temp\ipykernel_5456\1717203083.py:9: SettingWithCopyWarning

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
< 585,83.452500,83.932500,93.460000,96.610000,90.367500
585 - 630,81.900000,83.155000,87.132500,92.717500,81.417500
630 - 645,78.517500,81.625000,73.485000,84.392500,62.857500
645 - 680,76.996667,81.026667,66.163333,81.133333,53.526667


In [385]:
# find the maximumn and minimumn value of school size
max_budget = school_summary["Total Students"].max()
min_budget = school_summary["Total Students"].min()

# use the school summary but narrow it down to school size average math score, average reading score percent passing math percent passing reading perscent overall passing
score_by_size = school_summary[["Total Students","Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing"]]
score_by_size["Average Math Score"] = pd.to_numeric(score_by_size["Average Math Score"])
score_by_size["Average Reading Score"] = pd.to_numeric(score_by_size["Average Reading Score"])
score_by_size["% Passing Math"] = pd.to_numeric(score_by_size["% Passing Math"])
score_by_size["% Passing Reading"] = pd.to_numeric(score_by_size["% Passing Reading"])
score_by_size["% Overall Passing"] = pd.to_numeric(score_by_size["% Overall Passing"])



# bins for cutting
bins = [0,999.999,1999.999,5000]

# name for each bin
student_range = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# put values into the dataframe
score_by_size["School Size"] = pd.cut(score_by_size["Total Students"], bins, labels = student_range)
score_by_size = score_by_size.drop(columns = ["Total Students"])
score_by_size = score_by_size.groupby(["School Size"])
score_by_size = score_by_size.mean()
score_by_size

C:\Users\Peter\AppData\Local\Temp\ipykernel_5456\983260266.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_by_size["Average Math Score"] = pd.to_numeric(score_by_size["Average Math Score"])
C:\Users\Peter\AppData\Local\Temp\ipykernel_5456\983260266.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_by_size["Average Reading Score"] = pd.to_numeric(score_by_size["Average Reading Score"])
C:\Users\Peter\AppData\Local\Temp\ipykernel_5456\983260266.py:9: SettingWithCopyWarning: 
A value is tryin

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.820,83.92500,93.55000,96.10000,89.885
Medium (1000-2000),83.374,83.86600,93.59800,96.79000,90.620
Large (2000-5000),77.745,81.34375,69.96375,82.76625,58.285


In [386]:
# Perform the same operations as above, based on school type
score_by_type = school_summary[["School Type","Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing"]]
score_by_type["Average Math Score"] = pd.to_numeric(score_by_type["Average Math Score"])
score_by_type["Average Reading Score"] = pd.to_numeric(score_by_type["Average Reading Score"])
score_by_type["% Passing Math"] = pd.to_numeric(score_by_type["% Passing Math"])
score_by_type["% Passing Reading"] = pd.to_numeric(score_by_type["% Passing Reading"])
score_by_type["% Overall Passing"] = pd.to_numeric(score_by_type["% Overall Passing"])
score_by_type = score_by_type.groupby(["School Type"])
score_by_type = score_by_type.mean()
score_by_type


C:\Users\Peter\AppData\Local\Temp\ipykernel_5456\1016229391.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_by_type["Average Math Score"] = pd.to_numeric(score_by_type["Average Math Score"])
C:\Users\Peter\AppData\Local\Temp\ipykernel_5456\1016229391.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_by_type["Average Reading Score"] = pd.to_numeric(score_by_type["Average Reading Score"])
C:\Users\Peter\AppData\Local\Temp\ipykernel_5456\1016229391.py:5: SettingWithCopyWarning: 
A value is tr

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.472500,83.896250,93.620000,96.586250,90.431250
District,76.955714,80.965714,66.548571,80.798571,53.671429
